# 0.0 Imports

In [1]:
import pandas as pd
import numpy  as np

import inflection
import s3fs
import umap
import os

from sklearn import preprocessing as pp
from sklearn import cluster as c
from sklearn import metrics as m
from sklearn import decomposition as dd
from sklearn import manifold as mn
from sklearn import ensemble as en
from sklearn import mixture as mx
from sklearn.neighbors import NearestNeighbors

from scipy.cluster import hierarchy as hc

/home/ubuntu/.pyenv/versions/3.10.0/envs/cluster/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 0.1 Helper functions

In [2]:
def rename_columns(dataframe):
    df = dataframe.copy()
    title = lambda x: inflection.titleize(x)
    snakecase = lambda x: inflection.underscore(x)
    spaces = lambda x: x.replace(" ", "")
    cols_old = list(df.columns)
    #cols_old = list(map(title, cols_old))
    cols_old = list(map(spaces, cols_old))
    cols_new = list(map(snakecase, cols_old))
    df.columns = cols_new
    return df

## 0.2 Loading data

In [3]:
path_s3 = 's3://insiders-cluster'

In [4]:
df_raw = pd.read_excel(path_s3 + '/Online Retail.xlsx')

# 1.0 Data description

In [5]:
df1 = df_raw.copy()

## 1.1 Rename columns

In [6]:
# Using the function we created in section 0.1
df1 = rename_columns(df1)

## 1.5 Replace NA 

In [7]:
# Separando o df com os NA na coluna customer_id
df_missing = df1.loc[df1['customer_id'].isna(), :]
df_not_missing = df1.loc[~df1['customer_id'].isna(), :]

In [8]:
# Arbitrando um id para os todos os customers que estão sem id

# Definindo aleatoriamente para cada invoice unico (drop duplicate)
df_backup = pd.DataFrame(df_missing['invoice_no'].drop_duplicates())
df_backup['customer_id'] = np.arange(19000, 19000+len(df_backup), 1)

# merge original with reference dataframe
df1 = pd.merge( df1, df_backup, on='invoice_no', how='left' )

# coalesce 
df1['customer_id'] = df1['customer_id_x'].combine_first( df1['customer_id_y'] )

# drop extra columns
df1 = df1.drop( columns=['customer_id_x', 'customer_id_y'], axis=1 )

## 1.6 Change types

In [9]:
# invoice date
df1['invoice_date'] = pd.to_datetime( df1['invoice_date'], format='%d-%b-%y')

# customer id
df1['customer_id'] = df1['customer_id'].astype(int)

# invoice number
df1['invoice_no'] = df1['invoice_no'].astype(str)

# stock code
df1['stock_code'] = df1['stock_code'].astype(str)

# 2.0 Feature filtering

In [10]:
df2 = df1.copy()

In [11]:
# === Numerical attributes ====
df2 = df2.loc[df2['unit_price'] >= 0.04, :]

# === Categorical attributes ====
df2 = df2[~df2['stock_code'].isin( ['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY', 'DCGSSGIRL', 'PADS', 'B', 'CRUK'] )]

# description
df2 = df2.drop( columns='description', axis=1 )

# map 
df2 = df2[~df2['country'].isin( ['European Community', 'Unspecified' ] ) ]

# bad users
df2 = df2[~df2['customer_id'].isin( [16446] )]

# quantity
df2_returns = df2.loc[df1['quantity'] < 0, :]
df2_purchases = df2.loc[df1['quantity'] >= 0, :]

# 3.0 Feature engineering

In [12]:
df3 = df2.copy()

## 3.1 Feature creation

In [13]:
cols_drop = ['invoice_no', 'stock_code', 'quantity', 'invoice_date', 'unit_price', 'country']

df_ref = df3.drop(cols_drop, axis=1).drop_duplicates(ignore_index=True)

### 3.1.1 Gross Revenue

In [14]:
# Gross Revenue
df2_purchases['gross_revenue'] = df2_purchases['quantity'] * df2_purchases['unit_price']

# Monetary
df_monetary = df2_purchases[['customer_id', 'gross_revenue']].groupby('customer_id').sum().reset_index()
df_ref = pd.merge(df_ref, df_monetary, on='customer_id', how='left')

df_ref.isna().sum()

/tmp/ipykernel_33072/2433535372.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_purchases['gross_revenue'] = df2_purchases['quantity'] * df2_purchases['unit_price']


customer_id       0
gross_revenue    91
dtype: int64

### 3.1.2 Recency - Days from last purchase

In [15]:
# Recency - Last day purchase
# Calculate how many days has passed from the day that the customer had a purchase
# **inverter função**
df_recency = df2_purchases[['customer_id', 'invoice_date']].groupby( 'customer_id' ).max().reset_index()
df_recency['recency_days'] = ( df2_purchases['invoice_date'].max() - df_recency['invoice_date'] ).dt.days
df_recency = df_recency[['customer_id', 'recency_days']].copy()
#df_recency['recency_days'] = df_recency['recency_days'].apply( lambda x: 1 / x if x != 0 else 0)
df_ref = pd.merge( df_ref, df_recency, on='customer_id', how='left' )

df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
dtype: int64

### 3.1.3 Quantity of invoices (orders)

In [16]:
# Count of how many invoices the customer have
df_freq = (df2_purchases[['customer_id', 'invoice_no']].drop_duplicates()
                                                       .groupby( 'customer_id' )
                                                       .count()
                                                       .reset_index()
                                                       .rename(columns={'invoice_no':'qtd_invoice'}))
df_ref = pd.merge( df_ref, df_freq, on='customer_id', how='left' )
df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
qtd_invoice      91
dtype: int64

### 3.1.4 Quantity of total items bought

In [17]:
# Calculate how many items the customer has bought in total
df_qtd = df2_purchases.loc[:,['customer_id', 'quantity']].groupby('customer_id').sum().reset_index().rename(columns={'quantity':'qtd_items'})
df_ref = pd.merge( df_ref, df_qtd, on='customer_id', how='left' )
df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
qtd_invoice      91
qtd_items        91
dtype: int64

### 3.1.5 Quantity of unique products (stock_code/SKU)

In [18]:
df_stock = df2_purchases.loc[:, ['customer_id', 'stock_code']].drop_duplicates().groupby('customer_id').count().reset_index().rename(columns={'stock_code':'unique_products'})
df_ref = pd.merge( df_ref, df_stock, on='customer_id', how='left' )
df_ref.isna().sum()

customer_id         0
gross_revenue      91
recency_days       91
qtd_invoice        91
qtd_items          91
unique_products    91
dtype: int64

### 3.1.6 Average ticket value

In [19]:
# Avg Ticket
df_avg_ticket = df2_purchases[['customer_id', 'gross_revenue']].groupby( 'customer_id' ).mean().reset_index().rename( columns={'gross_revenue':'avg_ticket'} )
df_ref = pd.merge( df_ref, df_avg_ticket, on='customer_id', how='left')

df_ref.isna().sum()

customer_id         0
gross_revenue      91
recency_days       91
qtd_invoice        91
qtd_items          91
unique_products    91
avg_ticket         91
dtype: int64

### 3.1.7 Average receny days

In [20]:
df_aux = df2_purchases.loc[:,['customer_id', 'invoice_date']].drop_duplicates().sort_values(['customer_id', 'invoice_date'], ascending=[False, False])
df_aux['next_customer_id'] = df_aux['customer_id'].shift()
df_aux['previous_date'] = df_aux['invoice_date'].shift()

df_aux['avg_recency_days'] = df_aux.apply(lambda x: (x['previous_date'] - x['invoice_date']).days if x['customer_id']==x['next_customer_id'] else np.nan, axis=1)

df_aux = df_aux.drop(['invoice_date', 'next_customer_id', 'previous_date'], axis=1).dropna()

# Now we have how many days from last purchase, we will get the mean days between orders
df_aux = df_aux.groupby('customer_id').mean().reset_index()

df_ref = pd.merge( df_ref, df_aux, on='customer_id', how='left')

df_ref.isna().sum()

customer_id            0
gross_revenue         91
recency_days          91
qtd_invoice           91
qtd_items             91
unique_products       91
avg_ticket            91
avg_recency_days    2952
dtype: int64

### 3.1.8 Frequency purchase

In [21]:
df_aux = (df2_purchases.loc[:,['customer_id', 'invoice_no', 'invoice_date']].drop_duplicates()
                                                                            .groupby('customer_id')
                                                                            .agg(max_ = ('invoice_date','max'),
                                                                                 min_ = ('invoice_date','min'),
                                                                                 days_ = ('invoice_date', lambda x: (x.max()-x.min()).days + 1),
                                                                                 buy_ = ('invoice_no','count')
                                                                                )
                                                                            .reset_index())

# How many purchases per day the customer have
df_aux['frequency'] = df_aux[['buy_', 'days_']].apply(lambda x: x['buy_']/x['days_'] if x['days_'] != 0 else 0, axis=1)

df_aux = df_aux.drop(['max_', 'min_', 'days_', 'buy_'], axis=1)

df_ref = pd.merge(df_ref, df_aux, on='customer_id', how='left')

df_ref.isna().sum()

customer_id            0
gross_revenue         91
recency_days          91
qtd_invoice           91
qtd_items             91
unique_products       91
avg_ticket            91
avg_recency_days    2952
frequency             91
dtype: int64

### 3.1.9 Total returns

In [22]:
df_aux = df2_returns.loc[:, ['customer_id', 'quantity']].groupby('customer_id').sum().reset_index().rename(columns={'quantity':'qtd_returns'})
df_aux['qtd_returns'] = df_aux['qtd_returns'] *-1

df_ref = pd.merge( df_ref, df_aux, how='left', on='customer_id' )

# Input zero for customers that not have any returns
df_ref.loc[df_ref['qtd_returns'].isna(), 'qtd_returns'] = 0

df_ref.isna().sum()

customer_id            0
gross_revenue         91
recency_days          91
qtd_invoice           91
qtd_items             91
unique_products       91
avg_ticket            91
avg_recency_days    2952
frequency             91
qtd_returns            0
dtype: int64

### 3.1.10 Basket size - Quantity of items per order

In [23]:
df_aux = (df2_purchases.loc[:, ['customer_id', 'invoice_no', 'quantity']].groupby('customer_id')
                                                                         .agg(n_purchase=('invoice_no', 'nunique'), 
                                                                              n_products=('quantity', 'sum'))
                                                                         .reset_index())
# Calculate the average products bought per order
df_aux['avg_basket_size'] = df_aux['n_products'] / df_aux['n_purchase']

df_aux = df_aux.drop(columns=['n_purchase', 'n_products'], axis=1)

df_ref = pd.merge( df_ref, df_aux, how='left', on='customer_id' )
df_ref.isna().sum()

customer_id            0
gross_revenue         91
recency_days          91
qtd_invoice           91
qtd_items             91
unique_products       91
avg_ticket            91
avg_recency_days    2952
frequency             91
qtd_returns            0
avg_basket_size       91
dtype: int64

### 3.1.11 Unique basket size - Unique items per order

In [24]:
df_aux = (df2_purchases.loc[:, ['customer_id', 'invoice_no', 'stock_code']].groupby('customer_id')
                                                                         .agg(n_purchase=('invoice_no', 'nunique'), 
                                                                              n_products=('stock_code', 'nunique'))
                                                                         .reset_index())
# Calculate the average products bought per order
df_aux['avg_unique_basket_size'] = df_aux['n_products'] / df_aux['n_purchase']

df_aux = df_aux.drop(columns=['n_purchase', 'n_products'], axis=1)

df_ref = pd.merge( df_ref, df_aux, how='left', on='customer_id' )
df_ref.isna().sum()

customer_id                  0
gross_revenue               91
recency_days                91
qtd_invoice                 91
qtd_items                   91
unique_products             91
avg_ticket                  91
avg_recency_days          2952
frequency                   91
qtd_returns                  0
avg_basket_size             91
avg_unique_basket_size      91
dtype: int64

# 4.0 EDA - Análise exploratória dos dados

In [25]:
df4 = df_ref.dropna()
df4.isna().sum()

customer_id               0
gross_revenue             0
recency_days              0
qtd_invoice               0
qtd_items                 0
unique_products           0
avg_ticket                0
avg_recency_days          0
frequency                 0
qtd_returns               0
avg_basket_size           0
avg_unique_basket_size    0
dtype: int64

## 4.3 Estudo dos espaços

In [26]:
df43 = df4.drop(columns=['customer_id'], axis=1).copy()

### 4.3.4 Tree based embedding

In [27]:
# training dataset
X = df43.drop( columns=['gross_revenue'], axis=1 )
y = df43['gross_revenue']

# model definition
rf_model = en.RandomForestRegressor( n_estimators=100, random_state=42 )

# model training
rf_model.fit( X, y )

# predict
df_leaf = pd.DataFrame( rf_model.apply( X ) )

#### 4.3.4.1 UMAP embedding

In [28]:
# Reduzer dimensionality
reducer = umap.UMAP( random_state=42 )
embedding = reducer.fit_transform( df_leaf )

# embedding
df_tree = pd.DataFrame()
df_tree['embedding_x'] = embedding[:, 0]
df_tree['embedding_y'] = embedding[:, 1]

/home/ubuntu/.pyenv/versions/3.10.0/envs/cluster/lib/python3.10/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


# 5.0 Data Preparation

In [29]:
df5 = df4.copy()

# 6.0 Feature selection

In [30]:
df6 = df5.copy()

# 7.0 Hyperparameter Fine-Tunning

In [31]:
df7 = df_tree.copy()
X = df7.copy()

# 8.0 Machine Learning Modeling

## 8.1 K-Means

In [32]:
# Model definition
k = 10
kmeans = c.KMeans(n_clusters=k, n_init=10, random_state=42)

# model training
kmeans.fit( X )

# clustering
labels = kmeans.labels_

### 8.1.1 Cluster validation

In [33]:
## WSS ( Within-cluster sum of square)
print( 'WSS value: {}'.format( kmeans.inertia_ ) )

## SS ( Silhouette Score )
print( 'SS value: {}'.format( m.silhouette_score( X, labels, metric='euclidean' ) ) )

WSS value: 17574.89453125
SS value: 0.6398946642875671


# 9.0 Cluster analysis

## 9.2 Cluster profile

In [34]:
df92 = df4.copy()
df92['cluster'] = labels

In [35]:
# Number of customer
df_cluster = df92[['customer_id', 'cluster']].groupby( 'cluster' ).count().reset_index()
df_cluster['perc_customer'] = 100*( df_cluster['customer_id'] / df_cluster['customer_id'].sum() )

# Avg Gross revenue
df_avg_gross_revenue = df92[['gross_revenue', 'cluster']].groupby( 'cluster' ).mean().reset_index()
df_cluster = pd.merge( df_cluster, df_avg_gross_revenue, how='inner', on='cluster' )

# Avg recency days
df_avg_recency_days = df92[['recency_days', 'cluster']].groupby( 'cluster' ).mean().reset_index()
df_cluster = pd.merge( df_cluster, df_avg_recency_days, how='inner', on='cluster' )

# Avg invoice_no
df_qtde_products = df92[['qtd_invoice', 'cluster']].groupby( 'cluster' ).mean().reset_index()
df_cluster = pd.merge( df_cluster, df_qtde_products, how='inner', on='cluster' )

# Frequency
df_frequency = df92[['frequency', 'cluster']].groupby( 'cluster' ).mean().reset_index()
df_cluster = pd.merge( df_cluster, df_frequency, how='inner', on='cluster' )

# Returns
df_qtde_returns = df92[['qtd_returns', 'cluster']].groupby( 'cluster' ).mean().reset_index()
df_cluster = pd.merge( df_cluster, df_qtde_returns, how='inner', on='cluster' )

# 10.0 Deploy

In [36]:
df_cluster

,cluster,customer_id,perc_customer,gross_revenue,recency_days,qtd_invoice,frequency,qtd_returns
0,0,240,8.468596,1657.028292,32.095833,7.079167,0.031126,17.141667
1,1,386,13.620325,349.442383,100.994819,2.461140,0.201487,2.466321
2,2,249,8.786168,2566.144699,40.622490,7.305221,0.108972,20.477912
3,3,202,7.127735,969.124703,73.554455,2.613861,0.095760,11.628713
4,4,291,10.268172,15196.463333,20.398625,19.347079,0.102779,225.395189
5,5,433,15.278758,583.577460,79.498845,2.819861,0.107224,3.519630
6,6,343,12.103035,880.579971,70.317784,3.341108,0.094140,6.204082
7,7,189,6.669019,1913.410582,40.211640,5.253968,0.050315,22.201058
8,8,291,10.268172,3416.799622,31.154639,7.831615,0.040131,32.903780
9,9,210,7.410021,1449.856190,60.633333,3.042857,0.079146,7.823810


In [37]:
import sqlite3

In [38]:
from sqlalchemy import create_engine

In [39]:
# query
query_create_table_insiders = """
    CREATE TABLE insiders (
    cluster         INTEGER,
    customer_id     INTEGER,
    perc_customer   REAL,
    gross_revenue   REAL,
    recency_days    REAL,
    qtd_invoice     REAL,
    frequency       REAL,
    qtd_returns REAL
    )
"""
# database connection
host=os.environ.get('AWS_DB_HOST')
port=os.environ.get('AWS_DB_PORT')
database=os.environ.get('AWS_DB_DATABASE')
user=os.environ.get('AWS_DB_USER')
pwd=os.environ.get('AWS_DB_PWD')

# connection string
endpoint = f'postgresql://{user}:{pwd}@{host}:{port}/{database}'
conn = create_engine(endpoint)

# execute query
#conn.execute(query_create_table_insiders)


In [40]:
# insert data
df_cluster.to_sql('insiders', con=conn, if_exists='append', index=False)
#conn.close()

10